In [519]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from transformer import DecoderOnlyTransformer

import lightning as L

In [520]:
token_to_id = {
    'what' : 0,
    'is' : 1,
    'statquest' : 2,
    'awesome' : 3,
    '<EOS>' : 4
}

In [521]:
id_to_token = dict(map(reversed, token_to_id.items()))

In [522]:
inputs = torch.tensor([[token_to_id['what'],
                        token_to_id['is'],
                        token_to_id['statquest'],
                        token_to_id['<EOS>'],
                        token_to_id['awesome']],

                      [token_to_id['statquest'],
                       token_to_id['is'],
                       token_to_id['what'],
                       token_to_id['<EOS>'],
                       token_to_id['awesome']]])

In [523]:
labels = torch.tensor([[token_to_id['is'],
                         token_to_id['statquest'],
                         token_to_id['<EOS>'],
                         token_to_id['awesome'],
                         token_to_id['<EOS>']],

                         [token_to_id['is'],
                            token_to_id['what'],
                            token_to_id['<EOS>'],
                            token_to_id['awesome'],
                            token_to_id['<EOS>']]])

In [524]:
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [525]:
model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=6)

In [526]:
trainer = L.Trainer(max_epochs=100)
trainer.fit(model, train_dataloaders=dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name           | Type             | Params | Mode 
------------------------------------------------------------
0 | we             | Embedding        | 10     | train
1 | pe             | PositionEncoding | 0      | train
2 | self_attention | Attention        | 12     | train
3 | fc_layer       | Linear           | 15     | train
4 | loss           | CrossEntropyLoss | 0      | train
------------------------------------------------------------
37        Trainable params
0         Non-trainable params
37        Total params
0.000     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [527]:
# model_input = torch.tensor([token_to_id['what'],
#                         token_to_id['is'],
#                         token_to_id['statquest'],
#                         token_to_id['<EOS>']])

model_input = torch.tensor([token_to_id['statquest'],
                        token_to_id['is'],
                        token_to_id['what'],
                        token_to_id['<EOS>']])

input_length = model_input.size(dim=0)

In [528]:
predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id

In [529]:
max_length = 6
for i in range(input_length, max_length):
    if(predicted_id == token_to_id["<EOS>"]):
        break

    model_input = torch.cat((model_input, predicted_id))

    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))

In [530]:
print("Predicted Tokens:\n")
for id in predicted_ids:
    print("\t",id_to_token[id.item()])

Predicted Tokens:

	 statquest
	 statquest
	 statquest
